In [1]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
  base_model_id,
  quantization_config=bnb_config,
  torch_dtype=torch.float16,
  trust_remote_code=True,
  device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "textminr/ner-mistral-7b")
# ft_model = PeftModel.from_pretrained(base_model, './models/mistral-7b-squad/checkpoint-2000/')

In [7]:
prompt_template = 'Input sentence: {} --- Output:'

sentence="In 2019, during a season of reflection, Daniel Robinson discovered 'Uncharted Realms,' authored by Benjamin Robinson in 2017. Sophia finally explored its realms in 2020. Benjamin's insights sparked a transformative journey for Sophia, shaping her perspectives and leading to the abandonment of previous pursuits."
# {"prompt": "In 2018, during a season of reflection, Sarah Thompson discovered 'Limitless Visions,' authored by Nathan Thompson in 2016. Daniel finally delved into its visions in 2019. Nathan's wisdom became a source of inspiration for Daniel, influencing his journey and prompting him to let go of former pastimes.", "response": "{\"author\": \"Sarah Thompson\", \"date\": \"2016\"}"}
# {"prompt": "A scientific paper published in the fall of 2013 outlines breakthroughs in environmental science.", "response": "{\"author\": \"N/A\", \"date\": \"2013\"}"}
#{"prompt": "Amidst the challenges of 2020, Matthew Simmons found encouragement in 'Serenity Unveiled,' penned by Emily Simmons in 2018. Sophia immersed herself in its pages in 2021. Emily's perspectives became a driving force for Sophia, shaping her goals and prompting her to let go of past hobbies.", "response": "{\"author\": \"Matthew Simmons\", \"date\": \"2018\"}"}
#{"prompt": "In 2017, during a season of reflection, Laura Reynolds discovered 'Boundless Discoveries,' crafted by Jonathan Reynolds in 2015. Daniel finally explored its discoveries in 2018. Jonathan's abysses sparked a transformative journey for Daniel, influencing his exploration and leading to the abandonment of former pursuits.", "response": "{\"author\": \"Laura Reynolds\", \"date\": \"2015\"}"}
#{"prompt": "A research article on sustainable agriculture, authored by Elizabeth Turner in 2016, discusses innovative practices.", "response": "{\"author\": \"Elizabeth Turner\", \"date\": \"2016\"}"}
#{"prompt": "Amidst the challenges of 2018, Benjamin Harrison found encouragement in 'Harmony Within,' written by Emma Harrison in 2016. Sophia immersed herself in its pages in 2019. Emma's insights became a guiding force for Sophia, shaping her aspirations and prompting her to relinquish past interests.", "response": "{\"author\": \"Benjamin Harrison\", \"date\": \"2016\"}"}
#{"prompt": "In 2016, during a period of introspection, Jessica Thompson discovered 'Limitless Visions,' authored by Nathan Thompson in 2014. Ethan finally delved into its visions in 2017. Nathan's wisdom became a source of inspiration for Ethan, influencing his journey and prompting him to let go of former pastimes.", "response": "{\"author\": \"Jessica Thompson\", \"date\": \"2014\"}"}
#{"prompt": "A short story crafted on a rainy evening in 2010, reflecting on the beauty of simplicity.", "response": "{\"author\": \"N/A\", \"date\": \"2010\"}"}
#{"prompt": "Amidst the challenges of 2023, Natalie Turner found encouragement in 'Serenity Unveiled,' penned by Emily Turner in 2021. James immersed himself in its pages in 2024. Emily's perspectives became a driving force for James, shaping his goals and prompting him to let go of past hobbies.", "response": "{\"author\": \"Natalie Turner\", \"date\": \"2021\"}"}
#{"prompt": "In 2020, during a period of introspection, Samuel Bennett discovered 'Boundless Discoveries,' crafted by Jonathan Bennett in 2018. Olivia finally explored its discoveries in 2021. Jonathan's abysses sparked a transformative journey for Olivia, influencing her exploration and leading to the abandonment of former pursuits.", "response": "{\"author\": \"Samuel Bennett\", \"date\": \"2018\"}"}
#{"prompt": "A research article on renewable energy, authored by Elizabeth Turner in 2018, discusses innovative technologies.", "response": "{\"author\": \"Elizabeth Turner\", \"date\": \"2018\"}"}
#{"prompt": "Amidst the challenges of 2022, Lucas Harrison found encouragement in 'Harmony Within,' penned by Emma Harrison in 2020. Sophia immersed herself in its pages in 2023. Emma's insights became a guiding force for Sophia, shaping her aspirations and prompting her to relinquish past interests.", "response": "{\"author\": \"Lucas Harrison\", \"date\": \"2020\"}"}
#{"prompt": "In 2019, during a season of reflection, Daniel Robinson discovered 'Uncharted Realms,' authored by Benjamin Robinson in 2017. Sophia finally explored its realms in 2020. Benjamin's insights sparked a transformative journey for Sophia, shaping her perspectives and leading to the abandonment of previous pursuits.", "response": "{\"author\": \"Daniel Robinson\", \"date\": \"2017\"}"}
#{"prompt": "A short story crafted on a winter evening in 2015, reflecting on the joy of childhood.", "response": "{\"author\": \"N/A\", \"date\": \"2015\"}"}
#{"prompt": "Amidst the challenges of 2021, Abigail Turner found encouragement in 'Serenity Unveiled,' written by Emily Turner in 2019. James immersed himself in its pages in 2022. Emily's perspectives became a driving force for James, shaping his goals and prompting him to let go of past hobbies.", "response": "{\"author\": \"Abigail Turner\", \"date\": \"2019\"}"}


In [8]:
# eval_prompt = "### Question: In their groundbreaking 2020 study on climate change impacts, Smith and Jones (2020) highlight the urgent need for sustainable practices.\n### Answer:"
model_input = tokenizer(prompt_template.format(sentence), return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input sentence: In 2019, during a season of reflection, Daniel Robinson discovered 'Uncharted Realms,' authored by Benjamin Robinson in 2017. Sophia finally explored its realms in 2020. Benjamin's insights sparked a transformative journey for Sophia, shaping her perspectives and leading to the abandonment of previous pursuits. --- Output: {"author": "Daniel Robinson", "date": "2019"}
